## Telecom customers - churn model

In [31]:
# general
import sys
import os
import logging

# EDA
import pandas as pd
from ydata_profiling import ProfileReport
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns

# model
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import optuna
from imblearn.over_sampling import SMOTE, ADASYN
import shap
import joblib

# Make sure this points to the correct directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

# my packeges
from src.data_download import download_and_extract_kaggle_dataset
from src.preprocessing import target_variable_distribution, encode_binary_columns, encode_non_binary_columns,check_column_numeric,clean_column_names

# ignoring warnings
import warnings
warnings.filterwarnings("ignore")


### Data download from Kaggle

I created function which downloads a Kaggle dataset as a zip file into the specified folder (dataset) and extracts it.

In [ ]:
# variables
data_name = 'tarekmuhammed/telecom-customers'
data_path = 'dataset'

# download dataset from kaggle 
download_and_extract_kaggle_dataset(data_name, data_path)

### Exploratory data analysis (EDA)
 For EDA I use ProfileEReport, which generates and interactive, detailed report for an entire dataset with minimal code. It automatically performs a comprehensive Exploratory Data Analysis (EDA) and provides an overview of the data, helping you identify patterns, anomalies, and key statistics for further analysis. The report includes various insights such as missing values, distributions, correlations, and more.

In [ ]:
# load dataset 
df = pd.read_csv("./dataset/Telecom Customers Churn.csv")
df.head()

In [ ]:
# Generate the profile report
profile = ProfileReport(df, title="EDA", explorative=True)

# Display the report in a notebook 
profile.to_notebook_iframe()

# Save the report to an HTML file
profile.to_file("EDA_TEL.html")


Dataset Overview:

 - Number of variables (columns): 21
 - Number of observations (rows): 7043
 - Missing cells: There are no missing values, which is beneficial for data quality.
 - Duplicate rows: No duplicate rows, ensuring data integrity.

 Suggestions:
 - Feature Encoding: Most variables will need encoding (e.g., categorical to numeric) for model training.
 - Feature engineering: create new variables which could help the model make better predictions e.g StreamingService, ContractMonth, LongTermCustomer
 - Key Predictors for Churn: Based on the correlation matrix, features like Contract, TechSupport, tenure, and OnlineSecurity appear to be important predictors of churn.
 - Feature Redundancy: 
      - combine StreamingMovies and StreamingTV into a single feature (e.g., "StreamingService") 
      - drop PhoneService in favor of MultipleLines, which captures phone service status
      - change TotalCharges datatype from object to numeric 

### Preprocesing

##### Data Type Conversion

In [ ]:
# check column TotalCharge
print('The column TotalCharges has missing values, likely because the tenure is 0. We will fill these missing values with 0.')
check_column_numeric(df,'TotalCharges')

In [ ]:
# Convert 'TotalCharges' to numeric if it's an object or has non-numeric characters
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# fill 'TotalCharges' with 0 where 'tenure' is 0 and 'TotalCharges' is NaN
df.loc[(df['tenure'] == 0) & (df['TotalCharges'].isna()), 'TotalCharges'] = 0

df.info()

##### Feature Engineering

In [ ]:
# create new variables
df['StreamingService'] = np.where((df['StreamingTV'] == 'Yes') | (df['StreamingMovies'] == 'Yes'), 'Yes', 'No')
#df['AverageChargePerMonth'] = df['TotalCharges'] / (df['tenure'] + 1)
df['ContractMonth'] = df['tenure'] % 12
df['LongTermCustomer'] = np.where(df['tenure'] > 12, 1, 0)

##### Feature Encoding

In [ ]:
df = encode_non_binary_columns(df)

In [ ]:
df = encode_binary_columns(df)

#### Name conventions for columns

In [ ]:
# Clean column names
df = clean_column_names(df)

# Move 'Churn' to the first column
cols = ['churn'] + [col for col in df.columns if col != 'churn']

# Reorder the columns of the DataFrame
df = df[cols]

# Display the column names to check the order
df.columns

##### Removing features

In [ ]:
# Remove features with low variance
selector = VarianceThreshold(threshold=0.01)
X_new = selector.fit_transform(X)

# Drop highly correlated features
correlation_matrix = X.corr()
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.90:  # Threshold for correlation
            colname = correlation_matrix.columns[i]
            highly_correlated_features.add(colname)

X = X.drop(labels=highly_correlated_features, axis=1)
print(highly_correlated_features)

In [ ]:
# Remove unnecassary features 
customer_ids = df['customerID']
columns_to_drop = ['customerID','PhoneService', 'StreamingTV', 'StreamingMovies' ]

df = df.drop(columns_to_drop, axis=1)

### Model Building, Evaluation, Fine-tuning

In [ ]:
# target distribution
target_variable_distribution(df, target_col="churn")
print('The dataset has a clear imbalance between customers who did not churn (73.46%) and those who did churn (26.54%).')

In [ ]:
# split into training and testing sets
X = df.drop('churn', axis=1)
y = df['churn']  

# using parameter stratify especially when the dataset is imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Stacking model

- Concept: Stacking involves training multiple base models (e.g., CatBoost, XGBoost) and then using their predictions as inputs for a higher-level model (meta-model) like Logistic Regression.
- Goal: The meta-model combines the base models' predictions to improve overall performance.
- Strength: It leverages the strengths of different algorithms, often outperforming individual models by capturing diverse patterns in the data.



Optimal classification threshold: 0.349689581413797
Classification Report (After Threshold Tuning):
              precision    recall  f1-score   support

           0       0.89      0.75      0.81      1035
           1       0.52      0.73      0.60       374

    accuracy                           0.75      1409
   macro avg       0.70      0.74      0.71      1409
weighted avg       0.79      0.75      0.76      1409

Confusion Matrix:
[[779 256]
 [101 273]]
ROC AUC Score: 0.8213916660208221
True Negatives: 779, False Positives: 256, False Negatives: 101, True Positives: 273

Conclusion:
- The model shows solid overall performance but can be improved in predicting the positive class (class 1). The lower precision for class 1 suggests that there are many false positives, and the lower F1-score for class 1 indicates a potential trade-off between precision and recall.

In [ ]:

# Set up logging for better debugging
logging.basicConfig(level=logging.INFO)



# Features and target
X = df.drop(columns=['churn'])  # Replace 'churn' with your actual target column
y = df['churn']

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handle class imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define Optuna objective for CatBoost tuning
def catboost_objective(trial):
    param = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.3),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'eval_metric': 'AUC',
        'random_seed': 42,
        'verbose': 0
    }

    catboost_model = CatBoostClassifier(**param)
    score = cross_val_score(catboost_model, X_train_resampled, y_train_resampled, scoring='roc_auc', cv=5).mean()
    return score

# Define Optuna objective for XGBoost tuning
def xgboost_objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'use_label_encoder': False,
        'eval_metric': 'auc',
        'random_state': 42
    }

    xgb_model = XGBClassifier(**param)
    score = cross_val_score(xgb_model, X_train_resampled, y_train_resampled, scoring='roc_auc', cv=5).mean()
    return score

# Run Optuna for CatBoost
catboost_study = optuna.create_study(direction='maximize')
catboost_study.optimize(catboost_objective, n_trials=50)
catboost_best_params = catboost_study.best_params

# Run Optuna for XGBoost
xgboost_study = optuna.create_study(direction='maximize')
xgboost_study.optimize(xgboost_objective, n_trials=50)
xgboost_best_params = xgboost_study.best_params

# Stacking model with CatBoost and XGBoost
catboost_model = CatBoostClassifier(**catboost_best_params)
xgboost_model = XGBClassifier(**xgboost_best_params)

estimators = [
    ('catboost', catboost_model),
    ('xgboost', xgboost_model)
]

stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=CatBoostClassifier(verbose=0),  # or XGBoostClassifier
    cv=5
)

# Fit the stacking model on resampled data
stacking_model.fit(X_train_resampled, y_train_resampled)

# Predict probabilities and labels on the test set
y_pred_proba = stacking_model.predict_proba(X_test)[:, 1]

# Tune the classification threshold based on precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal classification threshold: {optimal_threshold}")

# Apply the tuned threshold to make predictions
y_pred_threshold_tuned = (y_pred_proba >= optimal_threshold).astype(int)

# Classification report after threshold tuning
print("Classification Report (After Threshold Tuning):")
print(classification_report(y_test, y_pred_threshold_tuned))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_threshold_tuned)
print(f"Confusion Matrix:\n{cm}")

# ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC Score: {roc_auc}")

# Detailed confusion matrix breakdown
tn, fp, fn, tp = cm.ravel()
print(f"True Negatives: {tn}, False Positives: {fp}, False Negatives: {fn}, True Positives: {tp}")


### XGBoost model

### Key Changes:
1. ADASYN Resampling: I replaced SMOTE with ADASYN to help in generating synthetic samples based on the difficulty of learning.
2. GridSearchCV: This is used to find the best hyperparameters, optimizing them over a range of values.
3. F1 Score for Threshold Selection: The optimal threshold is selected by maximizing the F1 score instead of a simple recall-precision trade-off.

- These steps will help balance the precision and recall, and potentially improve the performance on the imbalanced churn dataset.

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.8}
Optimal classification threshold: 0.26446035504341125
Confusion Matrix:
 [[747 288]
 [ 75 299]]
Classification Report (After Threshold Tuning):
              precision    recall  f1-score   support

           0       0.91      0.72      0.80      1035
           1       0.51      0.80      0.62       374

    accuracy                           0.74      1409
   macro avg       0.71      0.76      0.71      1409
weighted avg       0.80      0.74      0.76      1409

ROC AUC Score: 0.83291999276654

Conclusion:

This updated model demonstrates a better balance between precision and recall, especially in identifying churn cases. The recall for churn improved from 0.73 to 0.80, and the ROC AUC score also increased from 0.8214 to 0.8329. While there was a small trade-off in terms of precision for the churn class, the overall performance of the model improved, making it more effective in detecting churn.

Next Steps:

Further Threshold Tuning: I can experiment with additional threshold adjustments to maintain the higher recall while improving precision. This will help to fine-tune the model's prediction performance.
Cross-Validation: I plan to continue evaluating the model using cross-validation to ensure that the improvements generalize well across the entire dataset and are not limited to the test set.
Feature Importance/Analysis: I can investigate which features are contributing the most to these improvements. This will help to determine whether further feature engineering can push the model's performance even further.

In [ ]:


# Resample using ADASYN (alternative to SMOTE)
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

# Hyperparameter tuning with grid search for threshold optimization
param_grid = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [200, 500, 1000],
    'subsample': [0.6, 0.8, 1.0],
    'scale_pos_weight': [1, 3, 10]
}

# Use grid search for better hyperparameter selection
grid_search = GridSearchCV(XGBClassifier(use_label_encoder=False), param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train_resampled, y_train_resampled)

# Best hyperparameters found by grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the optimized XGBoost model
xgb_model = XGBClassifier(**best_params, use_label_encoder=False)
xgb_model.fit(X_train_resampled, y_train_resampled)

# Predict probabilities for threshold tuning
y_pred_proba_train = xgb_model.predict_proba(X_train)[:, 1]
y_pred_proba_test = xgb_model.predict_proba(X_test)[:, 1]

# Fine-tune classification threshold using precision-recall tradeoff
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_test)
pr_auc = auc(recall, precision)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

# Threshold selection: optimal threshold based on F1 score
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)  # F1 score maximization
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal classification threshold: {optimal_threshold}')

# Predict with the fine-tuned threshold
y_pred_adjusted = (y_pred_proba_test >= optimal_threshold).astype(int)

# Confusion matrix and evaluation metrics
cm = confusion_matrix(y_test, y_pred_adjusted)
print(f"Confusion Matrix:\n {cm}")

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Print classification report
print("Classification Report (After Threshold Tuning):")
print(classification_report(y_test, y_pred_adjusted))

# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba_test)
print(f"ROC AUC Score: {roc_auc}")


In [ ]:
# Save the model to a file
joblib.dump(xgb_model, 'xgb_model.pkl')

#### Cross - validation

Cross-Validation AUC Scores: [0.83459357 0.82601223 0.83685958 0.84436245 0.84650973]
Mean Cross-Validation AUC Score: 0.8376675120035433
Optimal classification threshold: 0.26446035504341125

Conclusion:

The model's cross-validation AUC scores demonstrate its strong discriminative ability, with a mean score of 0.84. The optimal classification threshold of 0.2645 is a crucial value for making predictions, and adjusting this threshold further can help fine-tune the balance between precision and recall.

In [ ]:
import joblib
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the saved model
xgb_model = joblib.load("xgb_model.pkl")

# Set up 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation to evaluate model performance
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='roc_auc')
print(f'Cross-Validation AUC Scores: {cv_scores}')
print(f'Mean Cross-Validation AUC Score: {np.mean(cv_scores)}')

# Predict probabilities for the test set (after cross-validation)
y_pred_proba_test = xgb_model.predict_proba(X_test)[:, 1]

# Fine-tune classification threshold using precision-recall tradeoff
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_test)
pr_auc = auc(recall, precision)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

# Threshold selection: optimal threshold based on F1 score
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)  # F1 score maximization
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal classification threshold: {optimal_threshold}')

# Predict with the fine-tuned threshold
y_pred_adjusted = (y_pred_proba_test >= optimal_threshold).astype(int)

# Confusion matrix and evaluation metrics
cm = confusion_matrix(y_test, y_pred_adjusted)
print(f"Confusion Matrix:\n {cm}")

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Print classification report
print("Classification Report (After Threshold Tuning):")
print(classification_report(y_test, y_pred_adjusted))

# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba_test)
print(f"ROC AUC Score: {roc_auc}")


### Some Ideas at the End

 - Cost-sensitive learning: Incorporate techniques that weigh the cost of misclassification differently for each class, helping to minimize costly false positives or negatives.
 - Gradient Boosting with Early Stopping: Prevent overfitting and improve generalization, especially for complex models.
 - Feature engineering: Explore creating new features from existing ones to provide the model with more meaningful data
 - Threshold tuning: Continue experimenting with different thresholds for classification to balance precision and recall based on business priorities.
 ...